In [1]:
import torch
from torch import nn
# from Teacher import Teacher_Model
from Teacher import Teacher_Model
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
import os
# from Utils_tool import EarlyStopping
# from Utils_tool_1 import load_data_drink
# from Utils_tool_1 import load_data_exercise
# from Utils_tool_1 import load_data_mimic_ii
import sys
# from transformers import AdamW
import torch.optim as optim
from datetime import datetime
import numpy as np
import h5py

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


class MyDataset(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, index):
        return torch.Tensor(np.array(self.x_data[index])), torch.Tensor(
            np.array(self.y_data[index]))

import plotly.graph_objects as go


def visualization(y1):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(y=y1,
                   mode="lines",
                   line=dict(color="blue"),
                   name="predict value"))
    fig.update_layout()
    fig.show()


def load_data_pulseDB(idx):
    x_tol_arr = []
    y_tol_arr = []
    for i in range(1, 11):
        # file_name = '/home/yonghu/data_yonghu/code_python/PulseDB_Ten_0728/Group%02d_in_out_split.h5' % i
        file_name = '/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group%02d_in_out_split.h5' % i
        print(file_name)
        with h5py.File(file_name, "r") as file:
            for dataset_name in file:
                if dataset_name=='In_Signals':
                    In_Signals = file[dataset_name][()][:,:,::2]
                    print(In_Signals.shape)
                    x_tol_arr.append(In_Signals)
                elif dataset_name=='Out_Signals':
                    Out_Signals = file[dataset_name][()][:,:,::2]
                    print(Out_Signals.shape)
                    y_tol_arr.append(Out_Signals)
            
    x_train_load_data = x_tol_arr[4:10] + x_tol_arr[0:2]
    x_val_load_data = x_tol_arr[3:4]
    x_test_load_data = x_tol_arr[2:3]
    x_train_load_data = np.vstack(x_train_load_data)
    x_val_load_data = np.vstack(x_val_load_data)
    x_test_load_data = np.vstack(x_test_load_data)
    print(x_train_load_data.shape)
    print(x_val_load_data.shape)
    print(x_test_load_data.shape)
    y_train_load_data = y_tol_arr[4:10] + y_tol_arr[0:2]
    y_val_load_data = y_tol_arr[3:4]
    y_test_load_data = y_tol_arr[2:3]
    y_train_load_data = np.vstack(y_train_load_data)
    y_val_load_data = np.vstack(y_val_load_data)
    y_test_load_data = np.vstack(y_test_load_data)
    print(y_train_load_data.shape)
    print(y_val_load_data.shape)
    print(y_test_load_data.shape)

    # y_test_load_data = y_test_load_data / 300

    # x_train_load_data = x_train_load_data[0:1000]
    # y_train_load_data = y_train_load_data[0:1000]
    # x_val_load_data = x_val_load_data[0:1000]
    # y_val_load_data = y_val_load_data[0:1000]
    # x_test_load_data = x_test_load_data[0:1000]
    # y_test_load_data = y_test_load_data[0:1000]

    x_train_data = torch.tensor(x_train_load_data)
    y_train_data = torch.tensor(y_train_load_data)
    x_val_data = torch.tensor(x_val_load_data)
    y_val_data = torch.tensor(y_val_load_data)
    x_test_data = torch.tensor(x_test_load_data)
    y_test_data = torch.tensor(y_test_load_data)

    train_data_set = MyDataset(x_train_data, y_train_data)
    val_data_set = MyDataset(x_val_data, y_val_data)
    test_data_set = MyDataset(x_test_data, y_test_data)

    return train_data_set, val_data_set, test_data_set

train_data_set, val_data_set, test_data_set = load_data_pulseDB("01")
import os
def create_folder_if_not_exists(folder_path):
    # 使用 os.path.exists() 判断文件夹是否存在
    if not os.path.exists(folder_path):
        # 如果文件夹不存在，则创建新的文件夹
        os.mkdir(folder_path)
        print(f"Folder '{folder_path}' created.\n")
    else:
        print(f"Folder '{folder_path}' already exists.\n")
        
dir_name = "VitalDB_DataLoader_tol/Folder03/"
create_folder_if_not_exists(dir_name)
torch.save(train_data_set, dir_name+"train_dataset.pt")
torch.save(val_data_set, dir_name+"val_dataset.pt")
torch.save(test_data_set, dir_name+"test_dataset.pt")


train_loader = DataLoader(train_data_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data_set, batch_size=64, shuffle=False)
print(len(train_loader), len(val_loader), len(test_loader))

cuda
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group01_in_out_split.h5
(125730, 4, 625)
(125730, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group02_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group03_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group04_in_out_split.h5
(125779, 4, 625)
(125779, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group05_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group06_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group07_in_out_split.h5
(125659, 4, 625)
(125659, 1, 625)
/home/yonghu/data_yonghu/code_python/VitalDB_dataset/VitalDB_Ten_0927/Group08_in_out_split.h5
(1256